In [1]:
# Import packages
import os
import urllib
import requests
import math
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import earthpy as et
import hydrofunctions as hf

# Date time conversion registration
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Get the data & set working director
# data = et.data.get_data('colorado-flood')
# os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

# Prettier plotting with seaborn
sns.set(font_scale=1.5, style="whitegrid")

In [2]:
# Create map of stations
hf.draw_map()

In [3]:
# # Request data for all stations in Colorado
# PR = hf.NWIS(stateCd='CO').get_data()

# # List the names for the first 5 sites in Colorado, USA
# PR.head()

In [4]:
# Define the site number and start and end dates that you are interested in
site = "06730500"
start = '1946-05-10'
end = '2018-08-29'

# Request data for that site and time period
longmont_resp = hf.get_nwis(site, 'dv', start, end)

# Convert the response to a json in order to use the extract_nwis_df function
longmont_resp = longmont_resp.json()

# Get metadata about the data
hf.get_nwis(site, 'dv').json()

Requested data from https://waterservices.usgs.gov/nwis/dv/?format=json%2C1.1&sites=06730500&startDT=1946-05-10&endDT=2018-08-29
Requested data from https://waterservices.usgs.gov/nwis/dv/?format=json%2C1.1&sites=06730500


{'name': 'ns1:timeSeriesResponseType',
 'declaredType': 'org.cuahsi.waterml.TimeSeriesResponseType',
 'scope': 'javax.xml.bind.JAXBElement$GlobalScope',
 'value': {'queryInfo': {'queryURL': 'http://waterservices.usgs.gov/nwis/dv/format=json%2C1.1&sites=06730500',
   'criteria': {'locationParam': '[ALL:06730500]',
    'variableParam': 'ALL',
    'parameter': []},
   'note': [{'value': '[ALL:06730500]', 'title': 'filter:sites'},
    {'value': '[mode=LATEST, modifiedSince=null]',
     'title': 'filter:timeRange'},
    {'value': 'methodIds=[ALL]', 'title': 'filter:methodId'},
    {'value': '2021-09-02T19:56:21.189Z', 'title': 'requestDT'},
    {'value': 'd82f2930-0c27-11ec-911f-005056beda50', 'title': 'requestId'},
    {'value': 'Provisional data are subject to revision. Go to http://waterdata.usgs.gov/nwis/help/?provisional for more information.',
     'title': 'disclaimer'},
    {'value': 'caas01', 'title': 'server'}]},
  'timeSeries': [{'sourceInfo': {'siteName': 'BOULDER CREEK AT MOUTH

In [5]:
# Get the data in a pandas dataframe format
longmont_discharge = hf.extract_nwis_df(longmont_resp)
l_discharge = pd.DataFrame(longmont_discharge[0])
l_discharge.head()

,USGS:06730500:00060:00003,USGS:06730500:00060:00003_qualifiers
datetimeUTC,,
1946-05-10 00:00:00+00:00,16.0,A
1946-05-11 00:00:00+00:00,19.0,A
1946-05-12 00:00:00+00:00,9.0,A
1946-05-13 00:00:00+00:00,3.0,A
1946-05-14 00:00:00+00:00,7.8,A


In [6]:

l_discharge.columns = ['discharge','flag']
l_discharge.head()

,discharge,flag
datetimeUTC,,
1946-05-10 00:00:00+00:00,16.0,A
1946-05-11 00:00:00+00:00,19.0,A
1946-05-12 00:00:00+00:00,9.0,A
1946-05-13 00:00:00+00:00,3.0,A
1946-05-14 00:00:00+00:00,7.8,A


In [7]:
l_discharge.tail()

,discharge,flag
datetimeUTC,,
2018-08-25 00:00:00+00:00,9.86,A
2018-08-26 00:00:00+00:00,7.02,A
2018-08-27 00:00:00+00:00,4.05,A
2018-08-28 00:00:00+00:00,2.67,A
2018-08-29 00:00:00+00:00,3.36,A


In [8]:
# Plot using matplotlib
fig, ax = plt.subplots(figsize=(11, 6))

ax.scatter(x=l_discharge.index.values,
           y=l_discharge["discharge"],
           marker="o",
           s=4,
           color="purple")

ax.set(xlabel="Date", ylabel="Discharge Value (CFS)",
       title="Stream Discharge - Station {} \n {} to {}".format(site, start, end))

plt.show()

/tmp/ipykernel_7910/452474490.py:13: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [38]:
# add a year column to your longmont discharge data
l_discharge["year"] = l_discharge.index.year

# Calculate annual max by resampling
l_discharge_annual_max = l_discharge.resample('AS').max()
l_discharge_annual_max.head()

,discharge,flag,year
datetimeUTC,,,
1946-01-01 00:00:00+00:00,99.000000,A,1946
1947-01-01 00:00:00+00:00,1930.000000,A,1947
1948-01-01 00:00:00+00:00,339.000000,A,1948
1949-01-01 00:00:00+00:00,2010.000000,hf.missing,1949
1950-01-01 00:00:00+00:00,63.594991,hf.missing,1950


In [39]:
# download usgs annual max data from figshare
url = "https://nwis.waterdata.usgs.gov/nwis/peak?site_no=06730500&agency_cd=USGS&format=rdb"
download_path = "/home/awstclair/github_forks/earth-analytics/data/colorado-flood/downloads/annual-peak-flow.txt"



urllib.request.urlretrieve(url, download_path)

('/home/awstclair/github_forks/earth-analytics/data/colorado-flood/downloads/annual-peak-flow.txt',
 <http.client.HTTPMessage at 0x7db83b3df2e0>)

In [40]:
# A function that counts the number of lines with a comment 
def count_the(file_url):
    r = requests.get(file_url, stream=True)
    file = r.text
    count = 0
    for line in file:
        if line.startswith('#'):
            count += 1
    return count

# Lines to skip
line_to_skip = count_the(url)+1

In [41]:
# Open the data using pandas
usgs_annual_max = pd.read_csv(download_path,
                              skiprows=[line_to_skip],
                              comment="#",
                              sep='\t',
                              usecols=["peak_dt","peak_va"],
                              parse_dates=["peak_dt"],
                              index_col="peak_dt")

usgs_annual_max.head()

,peak_va
peak_dt,
1927-07-29,407.0
1928-06-04,694.0
1929-07-23,530.0
1930-08-18,353.0
1931-05-29,369.0


In [42]:
# Add a year column to the data for easier plotting
usgs_annual_max["year"] = usgs_annual_max.index.year

# Are there any years that have two entries?
usgs_annual_max[usgs_annual_max.duplicated(subset="year") == True]

,peak_va,year
peak_dt,,
1947-10-15,721.0,1947
1993-10-18,497.0,1993


In [43]:
# Remove duplicate years - keep the max discharge value
usgs_annual_max = usgs_annual_max.sort_values(
    'peak_va', ascending=False).drop_duplicates('year').sort_index()

# If this returns no results you have successfully removed duplicates!
usgs_annual_max[usgs_annual_max.duplicated(subset="year") == True]

,peak_va,year
peak_dt,,


In [44]:
# Plot calculated vs USGS annual max flow values
fig, ax = plt.subplots(figsize=(11, 9))

ax.plot(usgs_annual_max["year"],
        usgs_annual_max["peak_va"],
        color="purple",
        linestyle=':',
        marker='o',
        label="Instantaneous Value")

ax.plot(l_discharge_annual_max["year"],
        l_discharge_annual_max["discharge"],
        color="lightgrey",
        linestyle=':',
        marker='o', label="Mean Daily Value")
ax.legend()
ax.set_title(
    "Annual Maxima - Downloaded Instantaneous vs. Derived Daily Peak Flows")

plt.show()

/tmp/ipykernel_10109/3179071256.py:20: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [ ]:
# https://www.earthdatascience.org/courses/use-data-open-source-python/use-time-series-data-in-python/floods-return-period-and-probability/

In [45]:
# Merge the two pandas dataframes on the year column
usgs_calculated = pd.merge(l_discharge_annual_max,
                           usgs_annual_max,
                           left_on="year",
                           right_on="year")

# Subtract usgs values from your calculated values
usgs_calculated["diff"] = usgs_calculated["peak_va"] - \
    usgs_calculated["discharge"]

In [46]:
usgs_calculated.head()

,discharge,flag,year,peak_va,diff
0,99.0,A,1946,178.0,79.0
1,1930.0,A,1947,2040.0,110.0
2,2010.0,hf.missing,1949,2020.0,10.0
3,1030.0,hf.missing,1951,1540.0,510.0
4,1460.0,A,1952,1990.0,530.0


In [47]:
# Sort data smallest to largest
longmont_discharge_sorted = l_discharge.sort_values(by="discharge")

# Count total obervations
n = longmont_discharge_sorted.shape[0]

# Add a numbered column 1 -> n to use in return calculation for rank
longmont_discharge_sorted.insert(0, 'rank', range(1, 1 + n))

# Calculate probability - note you may need to adjust this value based upon the time period of your data
longmont_discharge_sorted["probability"] = (
    (n - longmont_discharge_sorted["rank"] + 1) / (n + 1))
longmont_discharge_sorted["return-years"] = (
    1 / longmont_discharge_sorted["probability"])

In [49]:
longmont_discharge_sorted.tail()

,rank,discharge,flag,year,probability,return-years
datetimeUTC,,,,,,
2013-09-16 00:00:00+00:00,26406,3270.0,"A,e",2013,0.000189,5282.200000
2013-09-12 00:00:00+00:00,26407,3680.0,A,2013,0.000151,6602.750000
2013-09-15 00:00:00+00:00,26408,3970.0,"A,e",2013,0.000114,8803.666667
2013-09-14 00:00:00+00:00,26409,4970.0,"A,e",2013,0.000076,13205.500000
2013-09-13 00:00:00+00:00,26410,8910.0,"A,e",2013,0.000038,26411.000000


In [50]:

def calculate_return(df, colname):
    '''
    Add Documentation Here


    '''
    # Sort data smallest to largest
    sorted_data = df.sort_values(by=colname)
    
    # Count total obervations
    n = sorted_data.shape[0]
    
    # Add a numbered column 1 -> n to use in return calculation for rank
    sorted_data.insert(0, 'rank', range(1, 1 + n))
    
    # Calculate probability
    sorted_data["probability"] = (n - sorted_data["rank"] + 1) / (n + 1)
    
    # Calculate return - data are daily to then divide by 365?
    sorted_data["return-years"] = (1 / sorted_data["probability"])

    return(sorted_data)

In [53]:
longmont_prob = calculate_return(l_discharge, "discharge")
longmont_prob.head()

,rank,discharge,flag,year,probability,return-years
datetimeUTC,,,,,,
1954-08-10 00:00:00+00:00,1,0.0,A,1954,0.999962,1.000038
1946-06-25 00:00:00+00:00,2,0.0,A,1946,0.999924,1.000076
1946-07-05 00:00:00+00:00,3,0.0,A,1946,0.999886,1.000114
1954-08-08 00:00:00+00:00,4,0.1,A,1954,0.999849,1.000151
1955-04-24 00:00:00+00:00,5,0.2,A,1955,0.999811,1.000189


In [54]:

# Because these data are daily,
# divide return period in days by 365 to get a return period in years
longmont_prob["return-years"] = longmont_prob["return-years"] / 365
longmont_prob["probability"] = longmont_prob["probability"] * 365
longmont_prob.tail()

,rank,discharge,flag,year,probability,return-years
datetimeUTC,,,,,,
2013-09-16 00:00:00+00:00,26406,3270.0,"A,e",2013,0.06910,14.471781
2013-09-12 00:00:00+00:00,26407,3680.0,A,2013,0.05528,18.089726
2013-09-15 00:00:00+00:00,26408,3970.0,"A,e",2013,0.04146,24.119635
2013-09-14 00:00:00+00:00,26409,4970.0,"A,e",2013,0.02764,36.179452
2013-09-13 00:00:00+00:00,26410,8910.0,"A,e",2013,0.01382,72.358904


In [55]:
longmont_prob.head()


,rank,discharge,flag,year,probability,return-years
datetimeUTC,,,,,,
1954-08-10 00:00:00+00:00,1,0.0,A,1954,364.98618,0.00274
1946-06-25 00:00:00+00:00,2,0.0,A,1946,364.97236,0.00274
1946-07-05 00:00:00+00:00,3,0.0,A,1946,364.95854,0.00274
1954-08-08 00:00:00+00:00,4,0.1,A,1954,364.94472,0.00274
1955-04-24 00:00:00+00:00,5,0.2,A,1955,364.93090,0.00274


In [56]:
# Calculate the same thing using the USGS annual max data
usgs_annual_prob = calculate_return(usgs_annual_max, "peak_va")
usgs_annual_prob.head()

,rank,peak_va,year,probability,return-years
peak_dt,,,,,
1954-01-14,1,26.0,1954,0.985294,1.014925
1932-07-13,2,128.0,1932,0.970588,1.030303
1940-07-03,3,174.0,1940,0.955882,1.046154
1946-07-19,4,178.0,1946,0.941176,1.062500
2002-05-24,5,238.0,2002,0.926471,1.079365


In [57]:
# Compare both datasets
fig, ax = plt.subplots(figsize=(11, 6))

usgs_annual_prob.plot.scatter(x="peak_va",
                              y="probability",
                              title="Probability ",
                              ax=ax,
                              color='purple',
                              fontsize=16,
                              logy=True,
                              label="USGS Annual Max Calculated")

longmont_prob.plot.scatter(y="probability",
                           x="discharge",
                           title="Probability ",
                           ax=ax,
                           color='grey',
                           fontsize=16,
                           logy=True,
                           label="Daily Mean Calculated")
ax.legend(frameon=True,
          framealpha=1)

ax.set_ylabel("Probability")
ax.set_xlabel("Discharge Value (CFS)")
ax.set_title(
    "Probability of Discharge Events \n USGS Annual Max Data Compared to Daily Mean Calculated Annual Max")

plt.show()

/tmp/ipykernel_10109/1334902783.py:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
